**En este NoteBook se van a realizar actividades de limpieza extras**

In [1]:
import pandas as pd


# Cargar el archivo CSV
df_total_v4 = pd.read_csv("datos/datos_jugadores_v4/jugadores_total.csv")
df_total_v5 = pd.read_csv("datos/datos_jugadores_v5/jugadores_total.csv")

# Crear la nueva columna
df_total_v4['sufficient_minutes_played'] = df_total_v4['minutes_played'] >= 850
df_total_v5['sufficient_minutes_played'] = df_total_v5['minutes_played'] >= 850

In [2]:
df_mezcla = pd.read_csv("datos/normalizaciones_posicion/mezcla/mezcla_total.csv")
df_equipos = pd.read_csv("datos/datos_equipos/ligas_europeas16_equipos.csv")

In [3]:
# Supongamos que tu DataFrame se llama df_equipos
df_equipos.rename(columns={
    'team_id': 'team_id',
    'team': 'team_name',
    'presupuesto_fichajes': 'transfer_budget',
    'presupuesto_sueldos': 'wage_budget',
    'competicion': 'competition'
}, inplace=True)

In [4]:
df_mezcla.columns

Index(['Unnamed: 0', 'player_id', 'player_name', 'team', 'main_position',
       'team_id', 'minutes_played', 'saves_per90', 'save_percentage',
       'xg_against_minus_goals_conceded_per90', 'keeper_sweeper_per90',
       'penalty_save_percentage', 'aerial_dominance_index_per90',
       'pass_completion_rate', 'rating', 'rating_normalizado',
       'tackle_success_rate', 'tackles_successful_per90',
       'interceptions_per90', 'interception_success_rate', 'pressures_per90',
       'chances_created_per90', 'progressive_carries_per90',
       'crosses_completed_per90', 'duels_won_per90', 'fouls_committed_per90',
       'goals_scored_per90', 'goal_assists_per90', 'progressive_carries_rate',
       'clearances_per90', 'blocks_per90', 'times_dribbled_past_per90',
       'progressive_passes_per90', 'duel_success_rate',
       'ball_recoveries_per90', 'dribbles_completed_per90',
       'dribble_success_rate', 'key_passes_per90', 'shots_total_per90',
       'xg_total_per90', 'fouls_won_per90

Eliminar columnas innecesarias de los dataframes de equipos y estadísticas normalizadas

In [5]:
# 2. Eliminar columnas innecesarias

mezcla = ['player_id', 'saves_per90', 'save_percentage',
       'xg_against_minus_goals_conceded_per90', 'keeper_sweeper_per90',
       'penalty_save_percentage', 'aerial_dominance_index_per90',
       'pass_completion_rate',
       'tackle_success_rate', 'tackles_successful_per90',
       'interceptions_per90', 'interception_success_rate', 'pressures_per90',
       'chances_created_per90', 'progressive_carries_per90',
       'crosses_completed_per90', 'duels_won_per90', 'fouls_committed_per90',
       'goals_scored_per90', 'goal_assists_per90', 'clearances_per90',
       'blocks_per90', 'times_dribbled_past_per90', 'progressive_passes_per90',
       'duel_success_rate', 'ball_recoveries_per90',
       'dribbles_completed_per90', 'dribble_success_rate', 'key_passes_per90',
       'shots_total_per90', 'xg_total_per90', 'fouls_won_per90',
       'goals_minus_xg_per90', 'progressive_carries_rate', 'shot_accuracy',
       'penalties_won_per90', 'headed_shot_duel_rate']

columnas_a_eliminar = ['overall', 'potential', 'Unnamed: 0', 'outdated_rating']

# Asegúrate de que existen en el DataFrame antes de intentar eliminarlas
df_total_v4 = df_total_v4.drop(columns=[col for col in columnas_a_eliminar if col in df_total_v4.columns])
df_mezcla = df_mezcla[mezcla]

Corregir problema con aerial_dominance_index

In [6]:
for df in [df_total_v4, df_total_v5]:
    df['main_position'] = df['main_position'].astype(str)
    df.loc[df['main_position'] != 'goalkeeper', ['aerial_dominance_index', 'aerial_dominance_index_per90']] = 0


In [12]:
# Mostrar filas de df1 donde la posición no sea 'goalkeeper'
# para comprobar que los valores son 0
print(df_total_v4[df_total_v4['main_position'] == 'goalkeeper'][['player_name', 'main_position', 'aerial_dominance_index', 'aerial_dominance_index_per90']])


                           player_name main_position  aerial_dominance_index  \
6                          Mathew Ryan    goalkeeper                    13.5   
10                     Alphonse Areola    goalkeeper                    46.1   
26          Claudio Andrés Bravo Muñoz    goalkeeper                     9.3   
58    Francisco Guillermo Ochoa Magaña    goalkeeper                     7.8   
61                 Keylor Navas Gamboa    goalkeeper                    34.2   
...                                ...           ...                     ...   
1827                    Thomas Kessler    goalkeeper                     1.0   
1849           Mitchell James Langerak    goalkeeper                     2.7   
1851                        Jens Grahl    goalkeeper                     0.0   
1871               Alexander Manninger    goalkeeper                     0.0   
1884                      Dario Kresic    goalkeeper                     0.0   

      aerial_dominance_index_per90  
6 

In [20]:
# Comparar si los valores son diferentes
diferencias = df_total_v4[df_total_v4['aerial_dominance_index'] != df_total_v4['aerial_dominance_index_per90']]

# Mostrar las filas donde hay diferencia
print(diferencias[['player_name', 'main_position', 'aerial_dominance_index', 'aerial_dominance_index_per90']])

                           player_name main_position  aerial_dominance_index  \
6                          Mathew Ryan    goalkeeper                    13.5   
10                     Alphonse Areola    goalkeeper                    46.1   
26          Claudio Andrés Bravo Muñoz    goalkeeper                     9.3   
58    Francisco Guillermo Ochoa Magaña    goalkeeper                     7.8   
61                 Keylor Navas Gamboa    goalkeeper                    34.2   
...                                ...           ...                     ...   
1779                        Bernd Leno    goalkeeper                    43.0   
1784                      Thomas Kraft    goalkeeper                     3.7   
1813                    Felix Wiedwald    goalkeeper                    35.1   
1816                   Jaroslav Drobný    goalkeeper                    14.7   
1849           Mitchell James Langerak    goalkeeper                     2.7   

      aerial_dominance_index_per90  
6 

In [16]:
# Asegúrate de que no haya división por cero
df_total_v4['aerial_dominance_index_per90'] = df_total_v4.apply(
    lambda row: (row['aerial_dominance_index'] / row['minutes_played'] * 90)
    if row['minutes_played'] > 0 else 0,
    axis=1
)


In [17]:
# Hacemos merge para obtener la columna actualizada desde df_total_v4
merged = df_total_v5.merge(
    df_total_v4[['player_id', 'aerial_dominance_index_per90']],
    on='player_id',
    how='left',
    suffixes=('', '_from_v4')
)

# Reemplazar solo si el valor actual no es 0
merged['aerial_dominance_index_per90'] = merged.apply(
    lambda row: row['aerial_dominance_index_per90_from_v4']
    if row['aerial_dominance_index_per90'] != 0 else 0,
    axis=1
)

# Eliminar columna auxiliar y guardar cambios en df_total_v5
df_total_v5 = merged.drop(columns='aerial_dominance_index_per90_from_v4')


In [ ]:
# Guardar el resultado
df_total_v4.to_csv("datos/datos_jugadores_v4/jugadores_total.csv", index=False)
df_total_v5.to_csv("datos/datos_jugadores_v5/jugadores_total.csv", index=False)

In [ ]:
df_mezcla.to_csv("datos/normalizaciones_posicion/mezcla/mezcla_total.csv")
df_equipos.to_csv("datos/datos_equipos/ligas_europeas16_equipos.csv")